# Pendulum visualization using ipywidgets

version 01.3: includes lame animation

* Created 12-Dec-2018 by Dick Furnstahl (furnstahl.1@osu.edu)
* Last revised 26-Dec-2018 by Dick Furnstahl (furnstahl.1@osu.edu).

In [1]:
%matplotlib inline

In [2]:
import numpy as np
from scipy.integrate import ode, odeint

import matplotlib.pyplot as plt

## Pendulum code

In [3]:
class Pendulum():
    """
    Pendulum class implements the parameters and differential equation from 
     the diffeq_pendulum.cpp code.
     
    Parameters
    ----------
    omega0 : float
        natural frequency of the pendulum (\sqrt{g/l} where l is the pendulum length) 
    alpha : float
        coefficient of friction 
    f_ext : float
        amplitude of external force 
    omega_ext : float
        frequency of external force 
    phi_ext : float
        phase angle for external force 

    Methods
    -------
    dy_dt(y, t)
        Returns the right side of the differential equation in vector y, 
        given time t and the corresponding value of y.
    """
    def __init__(self,
                 omega0=1.,
                 alpha=0.2,
                 f_ext=0.2,
                 omega_ext=0.689,
                 phi_ext=0.
                ):
        self.omega0 = omega0
        self.alpha = alpha
        self.f_ext = f_ext
        self.omega_ext = omega_ext
        self.phi_ext = phi_ext
    
    def dy_dt(self, y, t):
        """
        This function returns the right-hand side of the diffeq: 
        [dtheta/dt d^2theta/dt^2]
        
        Parameters
        ----------
        y : float
            A 2-component vector with y[0] = theta(t) and y[1] = dtheta/dt
        t : float
            time 
            
        Returns
        -------
        
        """
        F_ext = self.f_ext * np.cos(self.omega_ext*t + self.phi_ext)
        return [y[1], -self.omega0**2*np.sin(y[0]) - self.alpha*y[1] + F_ext]

In [4]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, ax=None):
    """
    Return a figure axis with a plot of y vs. x. 
    """
    if ax is None:
        ax = plt.gca()

    ax.plot(x, y, label=label)
    if label is not None:
        ax.legend()
    if title is not None:
        ax.set_title(title)
    if axis_labels is not None:    
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax


## Interface using ipywidgets with interactive_output

In [5]:
# Import the widgets we will use (add more as needed!) 
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, Tab, Label, Checkbox
from ipywidgets import FloatSlider, IntSlider, Play, Dropdown, HTMLMath 

from IPython.display import display
from time import sleep

In [6]:
# This function generates the main output, which is a grid of plots
def pendulum_plots(theta_vs_time_plot=True, theta_dot_vs_time_plot=True, 
                   phase_space_plot=True, omega0=1., 
                   alpha=0.2, f_ext=0.2, omega_ext=0.689, phi_ext=0., 
                   theta0=0.8, theta_dot0=0.0, 
                   t_start=0, t_end=100, delta_t=0.1, plot_start=0,
                   animate_flag=False, t_index=0,
                   font_size=18):
    """
    Create plots for interactive_output according to the inputs.
    
    Based on generating a Pendulum instance and the requested graphs.
    
    Notes
    -----
        1. We generate a new Pendulum instance every time *and* solved
            the ODE every time, even if the only change is to parameters
            like t_start and t_end.  Should we care or is this just so
            cheap to recalculate that it doesn't matter?
            How could we structure this differently?
    """
    
    # add delta_t o it goes at least to t_end (probably should use linspace)
    t_pts = np.arange(t_start, t_end+delta_t, delta_t)  
        
    # Instantiate a pendulum with the passed (or default) values of the 
    #  natural frequency omega0, damping alpha, driving amplitude, frequency, 
    #  and phase (f_ext, omega_ext, phi_ext).
    #  Should we delete p1 at some point?  Is there a memory issue?
    p1 = Pendulum(omega0=omega0, alpha=alpha, f_ext=f_ext, omega_ext=omega_ext, 
                  phi_ext=phi_ext)
    y0 = [theta0, theta_dot0]  # initial conditions for the pendulum ODE

    # ODE solver parameters
    abserr = 1.0e-8
    relerr = 1.0e-6

    # For now we solve with odeint; give more options in the future.
    theta, theta_dot = odeint(p1.dy_dt, y0, t_pts,
                              atol=abserr, rtol=relerr).T
    
    # Update the common font size
    plt.rcParams.update({'font.size': font_size})
 
    # Labels for individual plot axes
    theta_vs_time_labels = (r'$t$', r'$\theta$')
    theta_dot_vs_time_labels = (r'$t$', r'$d\theta/dt$')
    phase_space_labels = (r'$\theta$', r'$d\theta/dt$')
    
    # Figure out how many rows and columns [one row for now]
    plot_flags = [theta_vs_time_plot, theta_dot_vs_time_plot, phase_space_plot]
    plot_num = plot_flags.count(True)
    plot_rows = 1
    figsize_rows = plot_rows*6
    plot_cols = plot_num
    figsize_cols = min(plot_cols*8, 16)  # at most 16
    
    # Make the plot!
    fig, axes = plt.subplots(plot_rows, plot_cols, 
                             figsize=(figsize_cols,figsize_rows))
    axes = np.atleast_1d(axes)  # make it always a 1d array, even if only 1

    start_index = (np.fabs(t_pts-plot_start)).argmin() # finds nearest index
    
    next_axes = 0
    if theta_vs_time_plot:
        plot_y_vs_x(t_pts, theta, axis_labels=theta_vs_time_labels, 
                    label='pendulum', title=r'$\theta$ vs. time', 
                    ax=axes[next_axes])    
        # add a line where the phase space plot starts
        axes[next_axes].axvline(t_pts[start_index], lw=3, color='red')
        if animate_flag:
            axes[next_axes].plot(t_pts[t_index], theta[t_index], 'ro')
        next_axes += 1
    
    if theta_dot_vs_time_plot:
        plot_y_vs_x(t_pts, theta_dot, axis_labels=theta_dot_vs_time_labels, 
                    label='pendulum', title=r'$d\theta/dt$ vs. time', 
                    ax=axes[next_axes])    
        # add a line where the phase space plot starts
        axes[next_axes].axvline(t_pts[start_index], lw=3, color='red')
        if animate_flag:
            axes[next_axes].plot(t_pts[t_index], theta_dot[t_index], 'ro')
        next_axes += 1

    if phase_space_plot:
        plot_y_vs_x(theta[start_index:-1], theta_dot[start_index:-1], 
                    axis_labels=phase_space_labels, title='Phase space', 
                    ax=axes[next_axes])    
        if animate_flag:
            axes[next_axes].plot(theta[t_index], theta_dot[t_index], 'ro')
        next_axes += 1
    
    fig.tight_layout()
    
    return fig, axes



In [7]:
# Widgets for the various inputs.
#   For any widget, we can set continuous_update=False if we don't want the 
#    plots to shift until the selection is finished (particularly relevant for 
#    sliders).

# Widgets for the plot choice (plus a label out front)
plot_choice_w = Label(value='Which plots: ',layout=Layout(width='100px'))
def plot_choice_widget(on=True, plot_description=None):
    """Makes a Checkbox to select whether to show a plot."""
    return Checkbox(value=on, description=plot_description,
                  disabled=False, indent=False, layout=Layout(width='150px'))
theta_vs_time_plot_w = plot_choice_widget(True, r'$\theta$ vs. time')
theta_dot_vs_time_plot_w = plot_choice_widget(False, r'$d\theta/dt$ vs. time')
phase_space_plot_w = plot_choice_widget(True, 'phase space')

# Widgets for the pendulum parameters (all use FloatSlider, so made function)
def float_widget(value, min, max, step, description, format):
    """Makes a FloatSlider with the passed parameters and continuous_update
       set to False."""
    slider_border = Layout(border='solid 1.0px')
    return FloatSlider(value=value,min=min,max=max,step=step,disabled=False,
                       description=description,continuous_update=False,
                       orientation='horizontal',layout=slider_border,
                       readout=True,readout_format=format)

omega0_w = float_widget(value=1.0, min=0.0, max=10., step=0.1,
                        description=r'natural $\omega_0$:', format='.1f')
alpha_w = float_widget(value=0.1, min=0.0, max=2., step=0.1,
                       description=r'damping $\alpha$:', format='.1f')
f_ext_w = float_widget(value=0.2, min=0.0, max=2., step=0.05,
                       description=r'strength $f_{\rm ext}$:', format='.2f')
omega_ext_w = float_widget(value=0.689,min=0.0,max=3.,step=0.1,
                       description=r'freq. $\omega_{\rm ext}$:', format='.2f')
phi_ext_w = float_widget(value=0.0, min=0, max=2.*np.pi, step=0.1,
                         description=r'phase $\phi_{\rm ext}$:', format='.1f')

# Widgets for the initial conditions
theta0_w = float_widget(value=0.8, min=0., max=2.*np.pi, step=0.1,
                        description=r'$\theta_0$:', format='.1f')
theta_dot0_w = float_widget(value=0.0, min=-10., max=10., step=0.1,
                            description=r'$(d\theta/dt)_0$:', format='.1f')

# Widgets for the plotting parameters
t_start_w = float_widget(value=0., min=0., max=100., step=10.,
                         description='t start:', format='.1f') 
t_end_w = float_widget(value=100., min=0., max=500., step=10.,
                       description='t end:', format='.1f')
delta_t_w = float_widget(value=0.1, min=0.01, max=0.2, step=0.01,
                         description='delta t:', format='.2f')
plot_start_w = float_widget(value=0., min=0., max=300., step=5.,
                            description='start plotting:', format='.1f')

# Widgets for the animating
animate_flag_w = Checkbox(value=False, description='Animate',
                  disabled=False, indent=False, layout=Layout(width='100px'))
t_index_w = Play(interval=100, value=0, min=0, max=1000, step=1, 
                      disabled=True, continuous_update=True,
                      description='press play', 
                      orientation='horizontal')

# Widgets for the styling parameters
font_size_w = Dropdown(options=['12', '16', '18', '20', '24'], value='18',
                       description='Font size:',disabled=False,
                       continuous_update=False,layout=Layout(width='140px'))

# Text for the help section in HTML (could move this to an external file!)
overview_text = \
   r"""<p>Here we explore the dynamics of a damped, driven pendulum. There is help 
          available under the other tabs.</p>  
          <ul>
            <li>Physics tab: find out about the equations being solved.
            <li>Plotting tab: adjust what is plotted and over what intervals.  
                Also about the algorithm used.
            <li>Styling tab: change how the plots look.
            <li>Animate tab: look at animated plots of the time dependence.
          </ul>      
    """ 
physics_text = \
   r"""<p>We have in mind a physical pendulum with moment of inertia $I$ for which 
       the dependent variable is the angle $\theta$.  It is subject to gravity, 
       a damping force proportional to $d\theta/dt \equiv \dot\theta$, and an external 
       periodic driving torque.  Newton's second law for the torque can be rearranged to
       give the differential equation:
       \begin{align}
         \qquad
         \frac{d^2\theta}{dt^2} + \alpha \frac{d\theta}{dt}
           + \omega_0^2 \sin\theta = f_{\rm ext}\cos(\omega_{\rm ext}t)
           \; .
       \end{align} 
       The presence of $\sin\theta$ rather than $\theta$ makes this problem
       inherently nonlinear.</p>
              
       <p>To study this system, we'll look at plots such as $\theta$ versus $t$ 
       and the phase-space plot $\dot\theta$ vs. $\theta$, given initial
       conditions $\theta_0 \equiv \theta(t=0)$ and 
       $\dot\theta_0 \equiv \dot\theta(t=0)$.
       Under what conditions will these look like the results for a
       simple harmonic oscillator?
       Try to decide what range of $\theta_0$ gives harmonic behavior, 
       given $\dot\theta_0 = 0$.</p>

       <p>Let's think about <em>chaos</em>.  Consider the following brief 
       discussion as a teaser.  Here are some characteristics of chaos:
       <ul>
         <li> The system does not repeat past behavior (cf.\ periodic
         behavior).</li>
         <li> An uncertainty (or variation) in initial conditions grows
         <em>exponentially</em> (rather than linearly) in time.  The consequence
         is that the system is deterministic (as opposed to having a random
         component) but not predictable, since there is a finite precision in
         specifying the initial conditions (e.g., think about round-off
         error).</li>
         <li> The system has a distributed power spectrum.</li>
       </ul>
       The following are necessary conditions for chaotic behavior:
       <ol type="a">
         <li> The system has at least \emph{three} independent dynamical
         variables.  That is, the system can be written as
         \begin{align}
            \qquad\frac{dy_0}{dt} &= F_0(y_0,\cdots,y_n) \ , \\
            \qquad\frac{dy_1}{dt} &= F_0(y_0,\cdots,y_n) \ , \\
                 &  \quad \vdots \\
            \qquad\frac{dy_n}{dt} &= F_0(y_0,\cdots,y_n) \ , \\
         \end{align}
         with $n \geq 3$.</li>
         <li>
           The equations of motion contain nonlinear term(s) that couple
           several of the variables.</li>
       </ol>
       You might not think that our pendulum example qualifies, since there
       only seem to be two independent dynamical variables, $\theta$ and
       $\omega \equiv \dot\theta$.  We find the third by introducing $\phi$
       as
       \begin{align}
         \qquad\phi &= \omega_{\rm ext} t \quad \Longrightarrow \quad
           \frac{d\phi}{dt} = \omega_{\rm ext} \ .
       \end{align}
       Thus, the three necessary equations are
       \begin{align}
         \qquad\frac{d\theta}{dt} &= \omega \ ,\\
         \qquad\frac{d\omega}{dt} &= -\alpha\omega - \omega_0^2 \sin\theta
            - f_{\rm ext}\cos\phi \ ,\\
         \qquad\frac{d\phi}{dt} &= \omega_{\rm ext} \ .
       \end{align}
       Now we satisfy a) with $\theta$, $\omega$, and $\phi$, and we satisfy
       b) since the $\sin\theta$ and $\cos\phi$ terms couple the equations
       nonlinearly.  So we should be able to find chaos!   
    """

plotting_text = \
    """
    <p>Notes on plotting:</p>
    <ul>
      <li>The <tt>plot_start</tt> variable sets when the phase space plot
       starts plotting.  This enables you to remove the transient behavior
       at early ties.
    </ul>
    """

styling_text = \
    """
    <p>For now you can only change the font size.</p>
    """

# Widgets for the help section, which are HTMLMath boxes in a Tab widget
help_max_height = '500px'
help_overview_w = HTMLMath(value=overview_text)
help_physics_w = HTMLMath(value=physics_text)
help_plotting_w = HTMLMath(value=plotting_text)
help_styling_w = HTMLMath(value=styling_text)
help_w = Tab(children=[help_overview_w, help_physics_w, 
                       help_plotting_w, help_styling_w], 
             layout=Layout(width='95%', max_height=help_max_height))
help_w.set_title(0, 'Overview')
help_w.set_title(1, 'Physics')
help_w.set_title(2, 'Plotting')
help_w.set_title(3, 'Styling')
help_w.set_title(4, 'Animate')

############## Begin: Explicit callback functions #######################

# Make sure that t_end is at least t_start + 50
def update_t_end(*args):
    if t_end_w.value < t_start_w.value:
        t_end_w.value = t_start_w.value + 50     
t_end_w.observe(update_t_end, 'value')
t_start_w.observe(update_t_end, 'value')


# Make sure that plot_start is at least t_start and less than t_end
def update_plot_start(*args):
    if plot_start_w.value < t_start_w.value:
        plot_start_w.value = t_start_w.value
    if plot_start_w.value > t_end_w.value:
        plot_start_w.value = t_end_w.value
plot_start_w.observe(update_plot_start, 'value')
t_start_w.observe(update_plot_start, 'value')
t_end_w.observe(update_plot_start, 'value')

# Only turn on play widget when animate is selected
def turn_on_play_widget(*args):
    if animate_flag_w.value is True:
        t_index_w.disabled = False
        t_pts = np.arange(t_start_w.value, t_end_w.value+delta_t_w.value, 
                          delta_t_w.value)  
        t_index_w.min = (np.fabs(t_pts-plot_start_w.value)).argmin() 
        t_index_w.max = len(t_pts) - 1
    else:
        t_index_w.disabled = True
animate_flag_w.observe(turn_on_play_widget, 'value')

############## End: Explicit callback functions #######################

# Set up the interactive_output widget 
plot_out = widgets.interactive_output(pendulum_plots,
                          dict(
                          theta_vs_time_plot=theta_vs_time_plot_w,
                          theta_dot_vs_time_plot=theta_dot_vs_time_plot_w,
                          phase_space_plot=phase_space_plot_w,
                          omega0=omega0_w,
                          alpha=alpha_w,
                          f_ext=f_ext_w,
                          omega_ext=omega_ext_w,
                          phi_ext=phi_ext_w,
                          theta0=theta0_w,
                          theta_dot0=theta_dot0_w,
                          t_start=t_start_w,
                          t_end=t_end_w, 
                          delta_t=delta_t_w,    
                          plot_start=plot_start_w, 
                          animate_flag=animate_flag_w,
                          t_index=t_index_w,
                          font_size=font_size_w)
                       )

# Now do some manual layout, where we can put the plot anywhere using plot_out
hbox1 = HBox([plot_choice_w, theta_vs_time_plot_w, theta_dot_vs_time_plot_w,
              phase_space_plot_w]) #  choice of what plots to show
hbox2 = HBox([omega0_w, f_ext_w, omega_ext_w, phi_ext_w])  # external driving parameters
hbox3 = HBox([theta0_w, theta_dot0_w, alpha_w]) # initial conditions and damping
hbox4 = HBox([t_start_w, t_end_w, delta_t_w, plot_start_w]) # time and plot ranges
hbox5 = HBox([font_size_w]) # font size
hbox6 = HBox([animate_flag_w, t_index_w])  # animate
hbox7 = HBox([help_w])  # help tabs

# We'll set up Tabs to organize the controls.  The Tab contents are declared
#  as tab0, tab1, ... (probably should make this a list?) and the overall Tab
#  is called tab (so its children are tab0, tab1, ...).
tab_height = '70px'  # Fixed minimum height for all tabs. Specify another way?
tab0 = VBox([hbox2, hbox3], layout=Layout(min_height=tab_height))
tab1 = VBox([hbox1, hbox4], layout=Layout(min_height=tab_height))
tab2 = VBox([hbox5], layout=Layout(min_height=tab_height))
tab3 = VBox([hbox6], layout=Layout(min_height=tab_height))
tab4 = VBox([hbox7], layout=Layout(min_height=tab_height))

tab = Tab(children=[tab0, tab1, tab2, tab3, tab4])
tab.set_title(0, 'Physics')
tab.set_title(1, 'Plotting')
tab.set_title(2, 'Styling')
tab.set_title(3, 'Animate')
tab.set_title(4, 'Help')

# Release the Kraken!
vbox2 = VBox([tab, plot_out])
display(vbox2)